# Overview of Paging Design Issues

Your kernel calls `kmalloc()` or you need to alloc page of userspace. Physical memory is full. You decide to evict a page.

* Who evicts the page?
    * allocator? kernel? page daemon?
* Which page to evict
    * obv a policy exists
    * dirty pages take longer
    * have to evict pages in other TLBs, have to follow the full stack

## Who evicts the page?

### Kernel scenario

Imagine this stack:
* `foo()`
* `kmalloc()`
* `getpages()` - **spinlock the coremap**
* evict, write to swap, wait on IO - **can't wait while holding spinlock!!**

This means that the kernel can't evict the page. You could have a spinlock anywhere in the stack.

> Recall if you took a spinlock, you've disabled interrupts. If you go wait on IO, you won't be able to receive the signal. OS161 asserts if you try to sleep with a spinlock.

This situation only occurs really if you realize that all pages are occupied and all pages are dirty. If a clean page existed, we can just chuck it and replace it with our own page. We can make this messy scenario more infrequent:
* have page daemon continually write dirty pages to disk (cleans them)
* ensuring the number of clean pages are above a certain threshold
* **reimplement stack allocation to get stack pages on demand. Otherwise you'll be deprived of memory so fast**
    * You'll need to do this anyway. Do this early on. Instead of allocating the stack right away, let the programs hit vm_faults 

### How else could kernel evict the page?
Delegate the IO wait to another thread. The kernel thread busywaits.

Obviously this sucks because busywaiting is trash.

### User scenario

Imagine a user program in this scenario:
* user program TBL fault
* kernel vm_fault, page not in memory, need clean page
* kernel spinlocks coremap

You're lucky here! You know the vm_fault could not have any spinlocks! You got here through the trap! 

This means you can just chuck the page writing to another thread.

You can set a busybit (eg. reference count) on a page and then release the coremap spinlock.

### Page Daemon scenario

Page daemon just hangs around and evicts things.

Same thing. Release the lock. Use the busybit to mark what you're transferring to swap.

### Summary

* Kernel scenario is most difficult
    * could have spinlocks held from the stack
    * no blocking inside kmalloc
    * we need to minimize the amount that this happens
* User and page daemon scenarios are simpler
    * can assume no spinlocks are held
    * synchronize the coremap using busybits to avoid locking it up while waiting for IO


## Which page to evict?

Dirty vs clean:
* **dirty** pages require IO, since changes have been made that haven't been saved
    * **saved** means it was recorded somewhere in the swapfile or in the executable
* **clean** could also be a zero page that hasn't been touched

Pages could be in a TLB or not in a TLB
* if a page to be evicted is mapped to a TLB, you have to evict the TLB entry
* this is complicated for a multiprocessor

Corresponding VM Region
* stack/heap/executable
    * heap and stack are likely to be dirty, executable likely to be clean
    * use a function that loads executables and then mark their pages as clean
    
How active is a page?

### In TLB vs not in TLB

**BEFORE** you evict a page, you must remove it from a TLB. Otherwise the other process is going to start overwriting your stuff!

On a multiprocessor, you have multiple TLBs. Then to do a **TLB shootdown**, you have to signal/interrupt the other processor, wait for them to remove it from the TLB, and then finally evict. 

This can be avoided. Give it some thought.

## Page replacement

How do we decide which pages to kick out when memory is tight.

How do we decide how much physical memory to allocate to a process?

### Paging and Swapping
Move things to swap.

* exploit locality
    * **temporal**: recently accessed memory is likely to be accessed again
    * **spatial**: memory near recently accessed memory is likely to be accessed soon
    * locality helps reduce frequency of paging

Ideally, we want to replace the pages that will never be used again.
* Farthest-in-the-future algorithm for caching lol

Exploiting this depends on many things
   * amount of locality and reference patterns in a program
   * page replacement policy
   * amount of physical mempory and the application footprint
   
### Basics
* free list could be a bitmap, one bit per page. Or could be a reference count

#### OPT/MIN: Farthest in Future
This is the optimal algorithm. However, this requires us to see into the future. So this is actually an impossible algorithm.

This is usually used as a yardstick to compare real algorithms against.

#### Random and FIFO
* Random: Throw out a random page
* FIFO: throw out pages in order they were allocated
    * sucky if you allocate and chuck
    
> Belady anomaly: increasing number of frames may increase number of page faults. Most commonly experienced when using FIFO page replacement algorithm

#### Least Recently Used
* Track when pages were referenced
* Use past behaviour to predict future behaviour (vs MIN who uses future information)

Using actual timestamps would eat up memory.

Instead:
* use 2-3 bit counter per page
* scan all pages in the system and increment the counters of unaccessed pages
* clear all reference bits for pages that have been accessed
* evict page with highest counter

#### LRU Clock
* *Clock hand* scans over all pages in the system
* There is a refbit, clear it as it goes along
* if it encounters a refbit = 0, evict the page

This saves you from having to go through the pages *again* to find the maximum 
